# Historical Trends

<b>01 - Increase total amount of service on the SHN and reliability of that service by 2024</b>

## Routes on the State Highway Network (SHN)
Transit routes along the SHN can be categorized into 3 groups:
1. **On SHN** - where at least 20% of the transit route runs the SHN (within 50 ft) 
2. **Intersects SHN** - where at least 35% of the transit route runs within 0.5 mile of the SHN.
3. **Other** - all other transit routes.

## Metrics
* service hours, service hours per route
* delay hours, delay hours per route

The metrics are shown for for transit routes **on the SHN** and **intersects SHN**.

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp.magics
import pandas as pd

import B1_report_metrics as report_metrics
from shared_utils import rt_dates 
from shared_utils import calitp_color_palette as cp
from bus_service_utils import chart_utils

In [5]:
quarterly_metrics_dict = {k: v for k, v in rt_dates.PMAC.items() if k != "Q1_2022"}

summary_df = report_metrics.concatenate_summary_across_dates(
    quarterly_metrics_dict, summary_dataset = "summary")

In [7]:
def base_quarterly_bar(df: pd.DataFrame, variable: str,
                       x_col: str, y_col: str) -> alt.Chart: 
    
    category = df.category.iloc[0]
    if category=="On SHN":
        color = cp.CALITP_CATEGORY_BRIGHT_COLORS[0]
    elif category=="Intersects SHN":
        color = cp.CALITP_CATEGORY_BRIGHT_COLORS[3]
    
    year = df.year.iloc[0]
    bar = (alt.Chart(df)
           .mark_bar()
           .encode(
               x=alt.X(f"{x_col}:O", axis=alt.Axis(format=f'{year} Q%q'), 
                       title = None),
               y=alt.Y(f"{y_col}:Q", title = chart_utils.labeling(variable)),
               color = alt.value(color),
               tooltip=['category', 
                        'year', 'quarter', 
                        'variable', 'value']
           )
          )
           
    return bar


def quarterly_bar_for_category(
    df: pd.DataFrame, 
    variable_list: list = ["service_hours", "delay_hours"], 
    category: str = "On SHN",
    x_col: str = "quarter(service_date)", 
    y_col: str = "value", 
    chart_height: int = 200, chart_width: int = 500,
) -> alt.Chart: 
    """
    Plot quarterly metrics within the same category.
    Ex: for all routes on SHN, show service hours, delay hours, 
    avg service hours, etc
    """
    subset = df[(df.variable.isin(variable_list)) & (df.category==category)]
   
    var1 = variable_list[0]
    df1 = subset[subset.variable==var1]
    var2 = variable_list[1]
    df2 = subset[subset.variable==var2]

    bar1 = (base_quarterly_bar(df1, var1, x_col, y_col)
            .properties(title={
                "text": f"{chart_utils.labeling(var1)}",
                "subtitle": f"Routes {category.title()}"
            }, width = chart_width, height = chart_height)
            .interactive()
           )
    
    bar2 = (base_quarterly_bar(df2, var2, x_col, y_col)
            .properties(title={
                "text": f"{chart_utils.labeling(var2)}",
                "subtitle": f"Routes {category.title()}"
            }, width = chart_width, height = chart_height)
            .interactive()
           )
    
    combined = (alt.hconcat(bar1, bar2)
                .resolve_scale(y="independent")
               )

    return combined

## Routes on SHN

In [8]:
# chart utils
HEIGHT = 250
WIDTH = 200

In [9]:
category = "On SHN"
var_list = ["service_hours_per_route", "delay_hours_per_route"]

chart = quarterly_bar_for_category(
    summary_df,
    variable_list = var_list, 
    category = category,
    x_col = "quarter(service_date)",
    y_col = "value",
    chart_height = HEIGHT, chart_width = WIDTH
)

chart

alt.HConcatChart(...)

In [10]:
var_list = ["service_hours", "delay_hours"]

chart = quarterly_bar_for_category(
    summary_df,
    variable_list = var_list, 
    category = category,
    x_col = "quarter(service_date)",
    y_col = "value",
    chart_height = HEIGHT, chart_width = WIDTH
)

chart

alt.HConcatChart(...)

## Routes Intersecting SHN

In [11]:
category = "Intersects SHN"
var_list = ["service_hours_per_route", "delay_hours_per_route"]

chart = quarterly_bar_for_category(
    summary_df,
    variable_list = var_list, 
    category = category,
    x_col = "quarter(service_date)",
    y_col = "value",
    chart_height = HEIGHT, chart_width = WIDTH
)

chart

alt.HConcatChart(...)

In [12]:
var_list = ["service_hours", "delay_hours"]

chart = quarterly_bar_for_category(
    summary_df,
    variable_list = var_list, 
    category = category,
    x_col = "quarter(service_date)",
    y_col = "value",
    chart_height = HEIGHT, chart_width = WIDTH
)

chart

alt.HConcatChart(...)

## Routes on SHN by District

In [13]:
district_df = report_metrics.concatenate_summary_across_dates(
    quarterly_metrics_dict, summary_dataset = "district")

In [93]:
def facet_by_district(df: pd.DataFrame, variable: str) -> alt.Chart:
    subset = df[df.variable == variable]
    
    bar = (alt.Chart(subset)
          .mark_bar()
          .encode(
              x=alt.X(f"quarter(service_date):O", 
                      axis=alt.Axis(format=f'2022 Q%q')),
              y = alt.Y("value:Q", title="Avg Service Hours"),
              color = alt.Color("District:N", title=None, 
                                scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS), legend=None),
              tooltip = ["District", "year_quarter", "value", "variable"]
          ).facet(facet="District:N", columns = 1, spacing=10, 
                  title = f"{variable.replace('_', ' ').title()}")
          .interactive()
         )
    
    return bar

In [96]:
bar1 = facet_by_district(district_df, "avg_service_hours")
bar2 = facet_by_district(district_df, "avg_delay_hours")

from shared_utils import styleguide
chart = styleguide.apply_chart_config(alt.hconcat(bar1, bar2))
chart

alt.HConcatChart(...)

In [91]:
#https://github.com/altair-viz/altair/issues/2177

var_list = ["avg_service_hours", "avg_delay_hours"]
subset = district_df = district_df[district_df.variable.isin(var_list)]

bar1 = (alt.Chart(subset[subset.variable=="avg_service_hours"])
 .mark_bar()
 .encode(
      x=alt.X(f"quarter(service_date):O", 
              axis=alt.Axis(format=f'2022 Q%q')),
     y = alt.Y("value:Q", title="Avg Service Hours"),
     color = alt.Color("District:N", title=None, 
                       scale = alt.Scale(
                           range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                           cp.CALITP_CATEGORY_BOLD_COLORS), 
                       legend=None),
     tooltip = ["District", "year_quarter", "value", "variable"]
 ).facet(facet="District:N", columns = 1, spacing=10, 
         title = "Avg Service Hours")
.interactive()
)

bar2 = (alt.Chart(subset[subset.variable=="avg_delay_hours"])
 .mark_bar()
 .encode(
      x=alt.X(f"quarter(service_date):O", 
              axis=alt.Axis(format=f'2022 Q%q')),
     y = alt.Y("value:Q", title="Avg Delay Hours"),
     color = alt.Color("District:N", title=None, 
                       scale = alt.Scale(
                           range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                           cp.CALITP_CATEGORY_BOLD_COLORS), 
                       legend=None),
     tooltip = ["District", "year_quarter", "value", "variable"]
 ).facet("District:N", columns = 1, spacing = 10, 
         title="Avg Delay Hours", 
        )
.interactive()
)

In [92]:
from shared_utils import styleguide
chart = (styleguide.apply_chart_config(alt.hconcat(bar1, bar2))
        )

chart

alt.HConcatChart(...)